In [4]:
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist
import time
import urllib
import os.path
import sys
import os
from os import listdir
from tensorflow.keras.utils import img_to_array, load_img
from numpy import asarray
from keras.preprocessing.image import ImageDataGenerator

#Data loading
x_train = []
y_train = []
x_test = []
y_test = []

def preprocess():
    folder_dir = "/Users/jcsansom/Documents/562 final/image_to_prompt/archive/Testing/glioma_tumor"
    for img in os.listdir(folder_dir):
        x_test.append(asarray(load_img(os.path.join(folder_dir, img))).tolist())
        y_test.append(0)   #0 = glioma tumor
    folder_dir = "/Users/jcsansom/Documents/562 final/image_to_prompt/archive/Testing/meningioma_tumor"
    for img in os.listdir(folder_dir):
        x_test.append(asarray(load_img(os.path.join(folder_dir, img))).tolist())
        y_test.append(1)   #1 = meningioma tumor
    folder_dir = "/Users/jcsansom/Documents/562 final/image_to_prompt/archive/Testing/no_tumor"
    for img in os.listdir(folder_dir):
        x_test.append(asarray(load_img(os.path.join(folder_dir, img))).tolist())
        y_test.append(2)   #2 = no tumor
    folder_dir = "/Users/jcsansom/Documents/562 final/image_to_prompt/archive/Testing/pituitary_tumor"
    for img in os.listdir(folder_dir):
        x_test.append(asarray(load_img(os.path.join(folder_dir, img))).tolist())
        y_test.append(3)   #3 = pituitary_tumor
    folder_dir = "/Users/jcsansom/Documents/562 final/image_to_prompt/archive/Training/glioma_tumor"
    for img in os.listdir(folder_dir):
        x_train.append(asarray(load_img(os.path.join(folder_dir, img))).tolist())
        y_train.append(0)   #0 = glioma tumor
    folder_dir = "/Users/jcsansom/Documents/562 final/image_to_prompt/archive/Training/meningioma_tumor"
    for img in os.listdir(folder_dir):
        x_train.append(asarray(load_img(os.path.join(folder_dir, img))).tolist())
        y_train.append(1)   #1 = meningioma tumor
    folder_dir = "/Users/jcsansom/Documents/562 final/image_to_prompt/archive/Training/no_tumor"
    for img in os.listdir(folder_dir):
        x_train.append(asarray(load_img(os.path.join(folder_dir, img))).tolist())
        y_train.append(2)   #2 = no tumor
    folder_dir = "/Users/jcsansom/Documents/562 final/image_to_prompt/archive/Training/pituitary_tumor"
    for img in os.listdir(folder_dir):
        x_train.append(asarray(load_img(os.path.join(folder_dir, img))).tolist())
        y_train.append(3)   #3 = pituitary_tumor
preprocess()


In [10]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_it = datagen.flow_from_directory('/Users/jcsansom/Documents/562 final/image_to_prompt/archive/Training/',
 class_mode='categorical', batch_size=10, target_size=(200, 200), color_mode="grayscale")
test_it = datagen.flow_from_directory('/Users/jcsansom/Documents/562 final/image_to_prompt/archive/Testing/',
 class_mode='categorical', batch_size=10, target_size=(200, 200), color_mode="grayscale")

Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [11]:

model = keras.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 1)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(4, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])



In [12]:
model.fit_generator(train_it, steps_per_epoch=len(train_it), validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=0)
model.summary()
score = model.evaluate_generator(test_it, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

/var/folders/rc/gp38btvd4t10gf8g1hgvk0s40000gn/T/ipykernel_7382/3048502482.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_it, steps_per_epoch=len(train_it), validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=0)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 200, 200, 32)      320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 100, 100, 32)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 100, 100, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 50, 50, 64)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 160000)            0         
                                                                 
 dense_4 (Dense)             (None, 128)              

/var/folders/rc/gp38btvd4t10gf8g1hgvk0s40000gn/T/ipykernel_7382/3048502482.py:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(test_it, verbose=0)


Test loss: 6.920057773590088
Test accuracy: 0.7411167621612549
